# Section 0: 실습 사전 준비
### 0.1. 실습 목표
이번 시간에는 AI Agent가 외부 시스템과 소통하는 핵심 기술인 **Tool Calling**과 **Structured Output**에 대해 깊이 있게 학습합니다. 수강생 여러분은 이번 실습을 통해 다음과 같은 역량을 확보하게 됩니다.

1.  **Tool Calling 원리 체득:** 사용자의 자연어 명령을 LLM이 분석하여, 시스템이 실행할 수 있는 구조화된 함수 호출(Function Call)로 변환하는 핵심 메커니즘을 이해합니다. 이는 Agent가 단순히 텍스트만 생성하는 것을 넘어, 실제 '행동'을 수행하게 만드는 첫걸음입니다.
2.  **병렬 처리(Parallel Calling) 구현:** "오늘 날씨와 주가 둘 다 알려줘"와 같은 사용자의 복합적인 질문에 대응하기 위해, 여러 함수를 동시에 호출하는 병렬 처리 방식을 구현합니다. 이를 통해 네트워크 지연 시간을 최소화하고 응답 속도를 향상시키는 실무적인 최적화 기법을 익힙니다.
3.  **구조화된 출력(Structured Output) 제어:** LLM의 자유로운 텍스트 출력을 Pydantic 모델을 활용하여 우리가 원하는 특정 JSON 구조로 '강제'하는 방법을 학습합니다. 이는 LLM의 응답을 다른 시스템(데이터베이스, 프론트엔드 등)과 안정적으로 연동하기 위한 필수 기술입니다.

### 0.2. 라이브러리 설치
이번 실습에 필요한 라이브러리들을 설치합니다. 각 라이브러리의 역할은 다음과 같습니다.

- `google-generativeai`: Google의 Gemini API를 Python 환경에서 사용하기 위한 공식 라이브러리입니다.
- `pydantic`: 데이터 유효성 검사 및 구조 정의를 위한 라이브러리입니다. LLM의 출력을 우리가 원하는 클래스(Class) 형태로 정확하게 변환하고 검증하는 데 사용됩니다.
- `instructor`: LLM의 출력을 Pydantic 모델로 강제하는 핵심 라이브러리입니다. 복잡한 후처리 코드 없이도 LLM의 응답을 신뢰할 수 있는 데이터 구조로 만들어줍니다.
- `finnhub-python`: 실시간 주식 데이터를 제공하는 Finnhub API를 쉽게 사용하기 위한 Python 클라이언트입니다.
- `arxiv`: arXiv.org의 논문 데이터를 프로그래밍 방식으로 검색하고 메타데이터를 가져올 수 있는 라이브러리입니다.

In [1]:
# !pip install google-generativeai "pydantic>=2.0" instructor finnhub-python arxiv jsonref -q

### 0.3. API 키 발급 및 보안 설정
이번 실습에서는 총 3개의 무료 API 키가 필요합니다.

**1. Google AI Studio API Key (LLM용):**
- [Google AI Studio](https://aistudio.google.com/app/apikey)에 접속하여 API 키를 생성합니다.
- `GOOGLE_API_KEY` 라는 이름으로 저장합니다

**2. OpenWeatherMap API Key (날씨 정보 Tool용):**
- [OpenWeatherMap](https://openweathermap.org/api)에 가입하고 로그인합니다.
- API Keys 탭에서 Default 키를 복사합니다.
- `OPENWEATHER_API_KEY` 라는 이름으로 저장합니다

**3. Finnhub API Key (주식 정보 Tool용):**
- [Finnhub](https://finnhub.io/register)에 가입하고 로그인합니다.
- 대시보드에서 API Key를 복사합니다. (무료 플랜으로 충분합니다.)
- `FINNHUB_API_KEY` 라는 이름으로 저장합니다

In [12]:
import os
import google.generativeai as genai
from getpass import getpass

# API 키 설정
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Google AI Studio API 키를 입력하세요: ")

if "OPENWEATHER_API_KEY" not in os.environ:
    os.environ["OPENWEATHER_API_KEY"] = getpass("OPENWEATHER_API_KEY 를 입력하세요: ")

if "FINNHUB_API_KEY" not in os.environ:
    os.environ["FINNHUB_API_KEY"] = getpass("FINNHUB_API_KEY 를 입력하세요: ")

# google-generativeai 라이브러리에 API 키를 설정합니다.
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

print("✅ 모든 API 키가 성공적으로 설정되었습니다.")

✅ 모든 API 키가 성공적으로 설정되었습니다.


# Section 1. 실제 외부 API를 활용한 Tool Calling 실습
### 1.1. Agent가 사용할 실제 Tool 정의
Agent가 외부 세계와 상호작용할 수 있도록, 실제 외부 API를 호출하는 두 개의 함수를 Tool로 정의합니다. 각 함수는 명확한 역할과 입출력 구조를 가집니다.

- `get_current_weather`: OpenWeatherMap API를 호출하여 특정 도시의 실시간 날씨 정보를 가져옵니다.
- `get_stock_price`: Finnhub API를 호출하여 특정 종목 코드의 현재 주가를 가져옵니다.

In [4]:
import json
import requests
import finnhub


def get_return_bigger(valueA: float, valueB: float) -> float:
    """두 수를 받아서 더 큰 수를 반환한다."""
    return max(valueA, valueB)


def get_current_weather(location: str, unit: str = "celsius") -> str:
    """OpenWeatherMap API를 사용하여 주어진 도시의 현재 날씨 정보를 가져옵니다."""
    api_key = os.environ["OPENWEATHER_API_KEY"]
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        weather_info = {
            "location": data["name"],
            "temperature": data["main"]["temp"],
            "unit": "celsius",
            "forecast": data["weather"][0]["description"],
        }
        return json.dumps(weather_info, ensure_ascii=False)
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP 오류: {http_err}"
    except Exception as e:
        return f"알 수 없는 오류: {e}"


def get_stock_price(symbol: str) -> str:
    """Finnhub API를 사용하여 주어진 종목 코드(symbol)의 현재 주가를 가져옵니다."""
    api_key = os.environ["FINNHUB_API_KEY"]
    finnhub_client = finnhub.Client(api_key=api_key)
    try:
        quote = finnhub_client.quote(symbol)
        if quote.get("c") is None or quote.get("c") == 0:
            return f"오류: '{symbol}'에 대한 주가 정보를 찾을 수 없습니다."
        stock_info = {"symbol": symbol, "price": quote["c"], "currency": "USD"}
        return json.dumps(stock_info)
    except Exception as e:
        return f"API 호출 중 오류 발생: {e}"

### 1.2. LLM을 위한 Tool 명세(Schema) 정의
LLM은 코드 자체를 이해하지 못합니다. 따라서 우리가 만든 함수를 LLM이 사용할 수 있도록, 각 함수의 기능과 사용법을 설명하는 '사용 설명서', 즉 **명세(Schema)**를 만들어 전달해야 합니다.

이 명세서의 `description` 필드가 얼마나 명확하고 상세한지에 따라 LLM의 Tool 선택 및 사용 능력이 크게 좌우됩니다. `google-generativeai` 라이브러리의 `Tool`과 `FunctionDeclaration`을 사용하여 이 명세를 체계적으로 정의합니다.

In [9]:
from google.generativeai.types import Tool, FunctionDeclaration

get_return_bigger_func = FunctionDeclaration(
    name="get_return_bigger",
    description="두 수를 받아서 더 큰 수를 반환한다.",
    parameters={
        "type": "object",
        "properties": {
            "valueA": {"type": "number", "description": "크기 비교할 첫번째 값"},
            "valueB": {"type": "number", "description": "크기 비교할 두번째 값"},
        },
        "required": ["valueA", "valueB"],
    },
)

get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="사용자가 지정한 도시의 현재 날씨 정보를 가져옵니다.",
    parameters={
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "날씨를 조회할 도시의 영어 이름 (예: Seoul, Busan)"},
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "온도 단위"},
        },
        "required": ["location"],
    },
)

get_stock_price_func = FunctionDeclaration(
    name="get_stock_price",
    description="미국 증시에 상장된 주식의 현재 가격을 티커 심볼(ticker symbol)을 사용하여 가져옵니다.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "주가를 조회할 주식의 티커 심볼 (예: Apple Inc.는 'AAPL', Google은 'GOOGL')",
            }
        },
        "required": ["symbol"],
    },
)

agent_tools = Tool(
    function_declarations=[
        get_return_bigger_func,
        get_current_weather_func,
        get_stock_price_func,
    ],
)

print("✅ Tool 명세가 성공적으로 정의되었습니다.")

✅ Tool 명세가 성공적으로 정의되었습니다.


### 1.3. Agent Executor 로직 구현
이제 LLM의 '제안'을 받아 실제로 함수를 실행하고, 그 결과를 다시 LLM에게 전달하여 최종 답변을 생성하는 **Agent의 핵심 실행 로직(Executor)**을 구현합니다. 이 2-Step 프로세스는 모든 Agent 시스템의 가장 기본적인 작동 원리입니다.

1.  **Suggestion (제안):** `사용자 질문` + `Tool 명세` → LLM → `함수 호출 제안(JSON)`
2.  **Execution (실행):** `함수 호출 제안(JSON)` → 개발자 코드(Executor) → `실제 함수 실행` → `실행 결과`
3.  **Final Answer (최종 답변):** `전체 대화 기록` + `실행 결과` → LLM → `최종 사용자 답변(자연어)`

In [11]:
model = genai.GenerativeModel(model_name="gemini-2.5-pro", tools=[agent_tools])


def agent_executor(prompt: str):
    print(f"🚀 사용자 질문: {prompt}")
    # Step 1: LLM의 '제안' 받기
    response = model.generate_content(prompt)
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"🧠 LLM의 제안: {function_call.name}({dict(function_call.args)})")

    # Step 2: 제안된 함수 '실행'
    available_functions = {
        "get_return_bigger": get_return_bigger,
        "get_current_weather": get_current_weather,
        "get_stock_price": get_stock_price,
    }
    if function_call.name in available_functions:
        function_to_call = available_functions[function_call.name]
        api_response = function_to_call(**function_call.args)
        print(f"🛠️ 함수 실행 결과: {api_response}")

        # Step 3: 실행 결과를 포함하여 최종 답변 요청
        conversation_history = [
            {"role": "user", "parts": [{"text": prompt}]},
            response.candidates[0].content,
            {
                "role": "function",
                "parts": [{"function_response": {"name": function_call.name, "response": {"content": api_response}}}],
            },
        ]
        response_final = model.generate_content(conversation_history)
        print(f"💬 최종 답변: {response_final.text}")
        return response_final.text
    else:
        return "오류: 모델이 제안한 함수를 찾을 수 없습니다."


# Agent 실행
agent_executor("1.1111과 1.9 중에 더 큰 수 뭐야?")

🚀 사용자 질문: 1.1111과 1.9 중에 더 큰 수 뭐야?
🧠 LLM의 제안: get_return_bigger_func({'valueB': 1.9, 'valueA': 1.1111})


'오류: 모델이 제안한 함수를 찾을 수 없습니다.'

# Section 2. 병렬 Tool Calling (Parallel Tool Calling)
실제 사용자는 여러 개의 의도를 하나의 질문에 담아 요청하는 경우가 많습니다. 이때 LLM이 각 의도를 파악하여, 필요한 여러 개의 함수 호출을 단 한 번의 요청으로 동시에 제안하는 기능이 바로 **병렬 Tool Calling**입니다. 이를 통해 불필요한 API 호출 횟수를 줄여 비용과 응답 시간을 최적화할 수 있습니다.

### 2.1. 병렬 실행을 위한 Agent Executor 개선
Section 1.3에서 만든 `agent_executor`를 개선하여, 모델이 여러 개의 함수 호출을 제안하더라도 모두 처리할 수 있는 `parallel_agent_executor`를 구현합니다. 핵심은 `response.candidates[0].content.parts`가 리스트 형태이므로, 이를 순회하며 모든 함수 호출을 실행하고 그 결과를 수집하는 것입니다.

In [8]:
def parallel_agent_executor(prompt: str):
    print(f"🚀 사용자 질문: {prompt}")
    # Step 1: LLM의 '제안' 받기
    response = model.generate_content(prompt)
    function_calls = response.candidates[0].content.parts
    print(f"🧠 LLM의 제안 ({len(function_calls)}개): ")
    for fc in function_calls:
        print(f"  - {fc.function_call.name}({dict(fc.function_call.args)})")

    # Step 2: 제안된 모든 함수 '실행'
    available_functions = {
        "get_current_weather": get_current_weather,
        "get_stock_price": get_stock_price,
    }
    api_responses = []
    for function_call in function_calls:
        function_name = function_call.function_call.name
        if function_name in available_functions:
            function_to_call = available_functions[function_name]
            api_response = function_to_call(**function_call.function_call.args)
            api_responses.append(api_response)
            print(f"🛠️ '{function_name}' 실행 완료: {api_response}")
        else:
            print(f"❌ 오류: '{function_name}' 함수를 찾을 수 없습니다.")

    # Step 3: 실행 결과들을 모두 포함하여 최종 답변 요청
    conversation_history = [
        {"role": "user", "parts": [{"text": prompt}]},
        response.candidates[0].content,
        {
            "role": "function",
            "parts": [
                {"function_response": {"name": fc.function_call.name, "response": {"content": resp}}}
                for fc, resp in zip(function_calls, api_responses)
            ],
        },
    ]
    response_final = model.generate_content(conversation_history)
    print(f"\n💬 최종 답변: {response_final.text}")
    return response_final.text


# 병렬 Agent 실행
parallel_agent_executor("1.2와 1.3과 1.111 를 큰 순서대로 알려줘")

🚀 사용자 질문: 1.2와 1.3과 1.111 를 큰 순서대로 알려줘
🧠 LLM의 제안 (3개): 
  - get_return_bigger({'valueB': 1.3, 'valueA': 1.2})
  - get_return_bigger({'valueB': 1.111, 'valueA': 1.3})
  - get_return_bigger({'valueB': 1.111, 'valueA': 1.2})
❌ 오류: 'get_return_bigger' 함수를 찾을 수 없습니다.
❌ 오류: 'get_return_bigger' 함수를 찾을 수 없습니다.
❌ 오류: 'get_return_bigger' 함수를 찾을 수 없습니다.


InvalidArgument: 400 Please use a valid role: user, model.

# Section 3. Structured Output: LLM 답변을 원하는 데이터 구조로 통제하기
Tool Calling이 LLM의 '행동'을 제어하는 기술이라면, **Structured Output**은 LLM의 '응답' 자체를 우리가 원하는 데이터 구조로 통제하는 강력한 기술입니다. 비정형 텍스트에서 정형 데이터를 추출하여 후속 자동화 작업에 활용하는 실무 시나리오에서 매우 중요합니다.

이번 실습에서는 6일차 최종 프로젝트와의 연계성을 고려하여, **arXiv의 논문 초록(비정형 텍스트)에서 논문 제목, 저자, 요약문 등 핵심 메타데이터(정형 데이터)를 정확하게 추출**하는 작업을 수행해 보겠습니다.

### 3.1. 데이터 구조를 표현할 Pydantic 모델 정의
LLM을 통해 비정형 텍스트에서 추출하고 싶은 정보의 구조를 Pydantic 모델(클래스)로 먼저 정의합니다. 이는 LLM에게 우리가 원하는 결과물의 '청사진'을 제공하는 것과 같습니다.

In [ ]:
from pydantic import BaseModel, Field
from typing import List


class ArxivPaperInfo(BaseModel):
    """arXiv 논문 초록에서 추출한 메타데이터를 담기 위한 데이터 구조입니다."""

    title: str = Field(..., description="논문의 전체 제목")
    authors: List[str] = Field(..., description="논문의 저자 목록")
    summary: str = Field(..., description="논문의 핵심 내용을 요약한 초록")
    primary_category: str = Field(..., description="논문의 주된 연구 분야 카테고리 (예: cs.CL, cs.AI)")

### 3.2. Instructor 클라이언트 설정 및 정보 추출
`instructor` 라이브러리는 기존 Gemini 클라이언트를 '패치(patch)'하여 `response_model`이라는 강력한 기능을 추가합니다. 이 파라미터에 우리가 방금 정의한 `ArxivPaperInfo` 클래스를 전달하면, LLM은 자신의 출력을 해당 Pydantic 모델의 JSON 스키마에 맞추어 생성하도록 강제됩니다.

In [ ]:
import instructor
import arxiv

# Instructor를 사용하여 Gemini 클라이언트를 확장합니다.
# mode=instructor.Mode.GEMINI_JSON을 명시하여 JSON 출력 모드를 활성화합니다.
client = instructor.from_provider(
    "google/gemini-2.5-pro",
    mode=instructor.Mode.GEMINI_JSON,
)

# 예시로 사용할 최신 AI 논문(Llama 3)의 초록을 arxiv API를 통해 가져옵니다.
paper = next(arxiv.Client().results(arxiv.Search(query="Llama 3")))
paper_abstract = paper.summary.replace("\n", " ")

print("--- 원본 논문 초록 (비정형 텍스트) ---")
print(paper_abstract)

# response_model에 ArxivPaperInfo를 지정하여, 출력을 해당 객체로 강제합니다.
paper_info_object = client.messages.create(
    messages=[
        {
            "role": "user",
            "content": f"다음 논문 초록 텍스트에서 핵심 정보를 추출해줘: {paper_abstract}",
        }
    ],
    response_model=ArxivPaperInfo,
)

print("\n" + "=" * 50)
print(f"반환된 객체의 타입: {type(paper_info_object)}")
print("--- 추출된 정보 (Pydantic 객체) ---")
print(paper_info_object)

### 3.3. 추출된 구조화 데이터의 활용
Instructor의 가장 큰 장점은 반환된 결과가 단순 텍스트가 아닌, **타입 검사가 완료된 신뢰할 수 있는 Pydantic 객체**라는 점입니다. 이를 통해 우리는 후속 코드에서 `.title`, `.authors` 와 같이 객체 지향적인 방식으로 안전하고 편리하게 데이터를 다룰 수 있습니다.

이렇게 추출된 구조화된 데이터는 데이터베이스에 저장하거나, 다른 API의 입력으로 사용하거나, 최종 보고서의 '참고문헌' 섹션을 자동으로 생성하는 등 다양한 자동화 작업에 즉시 활용될 수 있습니다.

In [ ]:
# Pydantic 객체의 각 속성에 직접 접근하여 데이터를 활용합니다.
print(f"논문 제목: {paper_info_object.title}")
print(f"주 저자: {paper_info_object.authors[0]}")
print(f"카테고리: {paper_info_object.primary_category}")

# 객체를 JSON으로 변환하여 파일로 저장하거나 다른 시스템으로 전송할 수 있습니다.
paper_info_json = paper_info_object.model_dump_json(indent=2)
print("\nJSON으로 변환된 결과:")
print(paper_info_json)